# Xarray tutorial

`Xarray` is an open source project and Python package that makes working with labelled multi-dimensional arrays simple, efficient, and fun!

## Install xarray
If you run code on your own computer, you need to install `xarray`. Open the console and enter ```pip install xarray``` or execute ```!pip install xarray``` is this jupyter notebook.

In [1]:
try:
    import xarray as xr
except ModuleNotFoundError:
    !pip install xarray
    import xarray as xr

print(xr.__version__)

0.19.0


In [2]:
import pandas as pd
import numpy as np

## Introduction to xarray Data Structures
Like `pandas`, `xarray` has two core data structures:
- `DataArray`
- `Dataset`

The `DataArray` is designed to hold a single multi-dimensional variable and its coordinates, and the `Dataset` is designed to hold multiple variables that potentially share the same coordinates.

## DataArray
DataArray has four important attributes:
* `values` is a array of `Numpy` and includes the data. 
* `dims` is a `tuple` that includes the names of all dimensions of `values` and its length is dimensions.
* `coords` is a dict-like built-in data structure of xarray and includes coordinate list or 1D-array of each dimension.
* `attrs` is a `dict` and you can use `attrs` to define some custom attributes.

### Defining a DataArray

You can use the `DataArray()` function to create a DataArray. There are 4 arguments corresponding to the four attributes above. The `data` argument is corresponding to `values` attribute. 

In [3]:
da = xr.DataArray(data=[[1,2,3],
                       [2,3,4]],
                   dims=['x','y'],
                   coords={'x': [10, 20],
                    'y': [10, 20, 30]},
                  attrs={'summary':'This is a custom DataArray for tutorial',
                        'license':'CC BY-NC-ND 4.0'})
da

<xarray.DataArray (x: 2, y: 3)>
array([[1, 2, 3],
       [2, 3, 4]])
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 10 20 30
Attributes:
    summary:  This is a custom DataArray for tutorial
    license:  CC BY-NC-ND 4.0

In [4]:
da.values

array([[1, 2, 3],
       [2, 3, 4]])

In [5]:
da.dims

('x', 'y')

In [6]:
da.coords

Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 10 20 30

In [7]:
da.attrs

{'summary': 'This is a custom DataArray for tutorial',
 'license': 'CC BY-NC-ND 4.0'}

You can specify the `data` argument to create a `DataArray`. By default, `dims` of `DataArray` is `('dim_0', 'dim_1', ...)` and `attrs` of `DataArray` is a empty `dict`.

In [8]:
da1 = xr.DataArray(data=[[1,2,3],
                       [2,3,4]])
da1

<xarray.DataArray (dim_0: 2, dim_1: 3)>
array([[1, 2, 3],
       [2, 3, 4]])
Dimensions without coordinates: dim_0, dim_1

In [9]:
da1.dims

('dim_0', 'dim_1')

In [10]:
da1.attrs

{}

In [11]:
da1.coords

Coordinates:
    *empty*

### Change attribute

In [12]:
da1 = da.copy(deep=True)
# change values
da1.values = np.random.rand(2,3)
# change dims
da1.rename(['m', 'n']) 
# add attrs
da1.attrs['gen'] = 'generated by numpy randomly'
# delete attrs
del da1.attrs['summary']
da1

<xarray.DataArray (x: 2, y: 3)>
array([[0.07008727, 0.19381815, 0.40912816],
       [0.18045071, 0.49072692, 0.12428202]])
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 10 20 30
Attributes:
    license:  CC BY-NC-ND 4.0
    gen:      generated by numpy randomly

## Datasets
Datasets has three important attributes:
1. `data_vars` is a `dict` with each key as the name of the variable or the dimension and each value as one of:
  - `1D-array` or `list`
  - `Series` of `pandas`
  - `DataFrame` of `pandas`
  - `tuple`  with two elements
  - `DataArray`

`1D-array` or `list`: When value of the `dict` is a 1D-array or list, the key of the `dict` is the name of the dimension and corresponding 1D-array or list is coordinates of the dimension.

In [13]:
import pandas as pd
ds = xr.Dataset(data_vars={'v1':    [1, 2, 3],
                   'v2': np.array([1, 2, 3])})
ds

<xarray.Dataset>
Dimensions:  (v1: 3, v2: 3)
Coordinates:
  * v1       (v1) int64 1 2 3
  * v2       (v2) int64 1 2 3
Data variables:
    *empty*

The `Series` of `pandas`: When value of the `dict` is a `Series` of `pandas`, the key of the `dict` is the name of the variable and values of the `Series` is values of the variable.  The `index` of the `Series` is values of coordinates of the variable.

In [14]:
ds = xr.Dataset(data_vars={'v1': pd.Series([1, 2, 3], index=['a', 'b', 'c'])})
ds

<xarray.Dataset>
Dimensions:  (dim_0: 3)
Coordinates:
  * dim_0    (dim_0) object 'a' 'b' 'c'
Data variables:
    v1       (dim_0) int64 1 2 3

In [15]:
s = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
s.index.name = 'str'
ds = xr.Dataset(data_vars={'v1': s})
ds

<xarray.Dataset>
Dimensions:  (str: 3)
Coordinates:
  * str      (str) object 'a' 'b' 'c'
Data variables:
    v1       (str) int64 1 2 3

The `DataFrame` of `pandas`: When value of the `dict` is a `DataFrame`, the key of the `dict` is the name of the variable and values of the `DataFrame` is values of the variable.  The `index` and `columns` of the `DataFrame` is values of coordinates of the variable. Therefore, the variable constructed by `DataFrame` has two dimensions.

In [16]:
df = pd.DataFrame([1, 2, 3], index=['a', 'b', 'c'])
df.index.name = 'str'
ds = xr.Dataset(data_vars={'v1': df})
ds

<xarray.Dataset>
Dimensions:  (str: 3, dim_1: 1)
Coordinates:
  * str      (str) object 'a' 'b' 'c'
  * dim_1    (dim_1) int64 0
Data variables:
    v1       (str, dim_1) int64 1 2 3

In [17]:
df = pd.DataFrame([[1, 2],
                  [2, 5], 
                  [3, 6]], index=['a', 'b', 'c'], columns=['c1', 'c2'])
df.index.name = 'str'
ds = xr.Dataset(data_vars={'v1': df})
ds

<xarray.Dataset>
Dimensions:  (str: 3, dim_1: 2)
Coordinates:
  * str      (str) object 'a' 'b' 'c'
  * dim_1    (dim_1) object 'c1' 'c2'
Data variables:
    v1       (str, dim_1) int64 1 2 2 5 3 6

`tuple` with two elements: When value of the `dict` is a `tuple`, the key of the `dict` is the name of the variable. The first elements of the `tuple` is name of dimension of the variable and second elements of the `tuple` is values of the variable.

In [18]:
tp = (('d1', 'd2'),[[1, 2],
      [2, 5], 
      [3, 6]])
ds = xr.Dataset(data_vars={'v1': tp})
ds

<xarray.Dataset>
Dimensions:  (d1: 3, d2: 2)
Dimensions without coordinates: d1, d2
Data variables:
    v1       (d1, d2) int64 1 2 2 5 3 6

`DataArray`:  When value of the `dict` is a `DataArray`, the key of the `dict` is the name of the variable. The dimensions and coordinates of the `DataArray` will be add to `Dataset` created.

In [19]:
ds = xr.Dataset(data_vars={'v1': da})
ds

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 10 20 30
Data variables:
    v1       (x, y) int64 1 2 3 2 3 4

2. `coords` also is a `dict`. The key of the `dict` is the name of the dimension and the values of the `dict` is the coordinate of the dimension.

In [20]:
ds = xr.Dataset(data_vars={'v1': tp},coords={'d1':['a', 'b', 'c'], 'd2':['c1', 'c2']})
ds

<xarray.Dataset>
Dimensions:  (d1: 3, d2: 2)
Coordinates:
  * d1       (d1) <U1 'a' 'b' 'c'
  * d2       (d2) <U2 'c1' 'c2'
Data variables:
    v1       (d1, d2) int64 1 2 2 5 3 6

3. `attrs` also is a `dict` that is same as the `attrs` of `DataArray`.

## Operations and Mathematical Functions

Data variables can be modified through Mathematical Functions which is same as array of `Numpy`.

In [21]:
da = xr.DataArray(data=[[1,2,3],
                       [2,3,4]],
                   dims=['x','y'],
                   coords={'x': [10, 20],
                    'y': [10, 20, 30]})
print(da*10)
print(np.log(da))

<xarray.DataArray (x: 2, y: 3)>
array([[10, 20, 30],
       [20, 30, 40]])
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 10 20 30
<xarray.DataArray (x: 2, y: 3)>
array([[0.        , 0.69314718, 1.09861229],
       [0.69314718, 1.09861229, 1.38629436]])
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 10 20 30


In [22]:
ds = xr.Dataset(data_vars={'v1': da})
print(ds*10)
print(np.log(ds))

<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 10 20 30
Data variables:
    v1       (x, y) int64 10 20 30 20 30 40
<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 10 20 30
Data variables:
    v1       (x, y) float64 0.0 0.6931 1.099 0.6931 1.099 1.386


## Loading Data from netCDF Files

NetCDF (Network Common Data Format) is a widely used format for distributing geoscience data. For more details about netCDF please access [netCDF website](https://www.unidata.ucar.edu/software/netcdf/docs/faq.html#whatisit)

Call `open_dataset` function open `netCDF` file. For more details about reading and writing netCDF files please access [xarray netCDF docs](http://xarray.pydata.org/en/latest/user-guide/io.html#netcdf). Below we download and load some ERA5 dataset.

In [23]:
# data source: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form
ds = xr.open_dataset('../../assets/data/era5_singapore_2021.9.1.nc')
ds

<xarray.Dataset>
Dimensions:    (longitude: 3, latitude: 3, time: 24)
Coordinates:
  * longitude  (longitude) float32 103.5 103.8 104.0
  * latitude   (latitude) float32 1.5 1.25 1.0
  * time       (time) datetime64[ns] 2021-09-01 ... 2021-09-01T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-10-01 12:00:45 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

## Selecting the Internal Data

You can directly access `Coordinates` and `Data variables` by their name.

In [24]:
ds.t2m

<xarray.DataArray 't2m' (time: 24, latitude: 3, longitude: 3)>
array([[[298.80975, 298.64566, 298.9777 ],
        [299.67108, 300.09094, 300.39368],
        [299.69064, 300.13788, 300.4328 ]],

       [[299.03745, 299.0257 , 299.40656],
        [299.81476, 299.89877, 300.39294],
        [299.74252, 300.01593, 300.48468]],

       ...,

       [[297.7106 , 297.41568, 297.7399 ],
        [299.78677, 299.86887, 300.029  ],
        [299.7594 , 299.7887 , 300.0837 ]],

       [[297.81107, 297.36383, 297.7017 ],
        [299.93607, 299.76227, 299.9537 ],
        [299.7818 , 299.71927, 300.03568]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 103.5 103.8 104.0
  * latitude   (latitude) float32 1.5 1.25 1.0
  * time       (time) datetime64[ns] 2021-09-01 ... 2021-09-01T23:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature

In [25]:
ds.time

<xarray.DataArray 'time' (time: 24)>
array(['2021-09-01T00:00:00.000000000', '2021-09-01T01:00:00.000000000',
       '2021-09-01T02:00:00.000000000', '2021-09-01T03:00:00.000000000',
       '2021-09-01T04:00:00.000000000', '2021-09-01T05:00:00.000000000',
       '2021-09-01T06:00:00.000000000', '2021-09-01T07:00:00.000000000',
       '2021-09-01T08:00:00.000000000', '2021-09-01T09:00:00.000000000',
       '2021-09-01T10:00:00.000000000', '2021-09-01T11:00:00.000000000',
       '2021-09-01T12:00:00.000000000', '2021-09-01T13:00:00.000000000',
       '2021-09-01T14:00:00.000000000', '2021-09-01T15:00:00.000000000',
       '2021-09-01T16:00:00.000000000', '2021-09-01T17:00:00.000000000',
       '2021-09-01T18:00:00.000000000', '2021-09-01T19:00:00.000000000',
       '2021-09-01T20:00:00.000000000', '2021-09-01T21:00:00.000000000',
       '2021-09-01T22:00:00.000000000', '2021-09-01T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2021-09-01 ... 2021-09-01T23:00:00
Attributes:
    long_name:  time

In [26]:
ds.longitude

<xarray.DataArray 'longitude' (longitude: 3)>
array([103.5 , 103.75, 104.  ], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 103.5 103.8 104.0
Attributes:
    units:      degrees_east
    long_name:  longitude

Below you can access the specific value by the number index or slices

In [27]:
# first value of time dimension
ds.time[0]

<xarray.DataArray 'time' ()>
array('2021-09-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2021-09-01
Attributes:
    long_name:  time

In [28]:
# get a 2-demension data with first time index 
ds.t2m[0]

<xarray.DataArray 't2m' (latitude: 3, longitude: 3)>
array([[298.80975, 298.64566, 298.9777 ],
       [299.67108, 300.09094, 300.39368],
       [299.69064, 300.13788, 300.4328 ]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 103.5 103.8 104.0
  * latitude   (latitude) float32 1.5 1.25 1.0
    time       datetime64[ns] 2021-09-01
Attributes:
    units:      K
    long_name:  2 metre temperature

You also can use `sel()` function to use label-based indexing.

In [29]:
ds.t2m.sel(time=ds.time[0])

<xarray.DataArray 't2m' (latitude: 3, longitude: 3)>
array([[298.80975, 298.64566, 298.9777 ],
       [299.67108, 300.09094, 300.39368],
       [299.69064, 300.13788, 300.4328 ]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 103.5 103.8 104.0
  * latitude   (latitude) float32 1.5 1.25 1.0
    time       datetime64[ns] 2021-09-01
Attributes:
    units:      K
    long_name:  2 metre temperature

In [30]:
#  compute the average value in 2021-09-01 00:00:00
ds.t2m[0].values.mean()

299.65002

## References
+ [Xarray Fundamentals of Earth and Enrivonmental Data Science](https://earth-env-data-science.github.io/lectures/xarray/xarray.html).
+ [Xarray documentation](http://xarray.pydata.org/en/latest/index.html)